In [97]:
import numpy as np
import pandas as pd
import re
import nltk
import pickle
from nltk.tokenize import sent_tokenize, word_tokenize

In [110]:
#word = "yooks"
#sample_text = [
#"yooks are not zooks.",
#"yooks know how to cook.",
#"yooks are from yooktown.",
#"yooks are not from zooktown",
#"yooks like to cook.",
#"yooks like to read books.",
#"yooks like to read magazines.",
#"there are no yooks in zooktown."
#]

word = "markov"
with open('matches.pkl', 'rb') as f:
  sample_text = pickle.load(f)

In [129]:
sample_text[3]

'markov models used in different situations, depending on whether every sequential state is observable or not, and whether the system is to be adjusted on the basis of observations made:'

In [112]:
pattern = re.compile(word)

In [113]:
iters = [pattern.finditer(text) for text in sample_text]

In [114]:
tokenized_text = [word_tokenize(text) for text in  sample_text]

In [147]:
tokenized_text[17]

['markov',
 'models',
 'can',
 'be',
 'applied',
 'to',
 'categorize',
 'human',
 'behavior',
 'at',
 'various',
 'levels',
 'of',
 'abstraction']

In [118]:
word_indices = [text.index(word) for text in tokenized_text]

In [148]:
word_indices

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [168]:
raw_tree = {"name": word}
raw_tree["raw_children"] = [text[i+1:] for i,text in zip(word_indices,tokenized_text)]

In [169]:
def process_child(child):
  final = {"name": child[0]}
  if len(child)==1:
    final["size"]=1
  else:
    final["children"] = [process_child(child[1:])]
  return final
raw_tree["children"] = [process_child(child) for child in raw_tree["raw_children"]]

In [170]:
def combine_children(children, depth=0):
  new_children = {}
  for child in children:
    if child["name"] in new_children:
      new_children[child["name"]].get("children")
      if child.get("children") and new_children[child["name"]].get("children"):
        new_children[child["name"]]["children"] = combine_children(list(child["children"])+ list(new_children[child["name"]]["children"]), depth=depth+1)
      elif child.get("children"):
        new_children[child["name"]]["children"] = child["children"]
      new_children[child["name"]]["size"] = new_children[child["name"]].get("size",0) + child.get("size",0)
    else:
      new_children[child["name"]] = child
  return list(new_children.values())

In [171]:
raw_tree["children"] = combine_children(raw_tree["children"])

In [172]:
def fold_tree(tree):
  if not tree.get("children"):
    return tree
  for child in tree["children"]:
    child = fold_tree(child)
  if len(tree["children"])==1:
    tree["name"] = tree["name"] + " " + tree["children"][0]["name"]
    tree["size"] = tree.get("size",0) + tree["children"][0].get("size",0)
    tree["children"] = tree["children"][0].get("children",None)
  return tree
  

In [173]:
raw_tree = fold_tree(raw_tree)

In [174]:
new_tree = raw_tree.copy()
new_tree["raw_children"] = None

In [175]:
import json
with open('tree.json', 'w') as outfile:
    json.dump(new_tree, outfile, indent=2, sort_keys=True)

In [176]:
subitem = [item["children"] for item in raw_tree["children"] if item["name"]=="models"] 
subitem[0]

[{'children': None,
  'name': 'are the hierarchical hidden markov model [ 3 ] and the abstract hidden markov model',
  'size': 1},
 {'children': None,
  'name': 'can be applied to categorize human behavior at various levels of abstraction',
  'size': 1},
 {'name': 'exist', 'size': 1},
 {'children': None,
  'name': 'used in different situations , depending on whether every sequential state is observable or not , and whether the system is to be adjusted on the basis of observations made :',
  'size': 1}]

SyntaxError: invalid syntax (<ipython-input-177-fa5b38373027>, line 6)